In [121]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('csv/kc_crimes_part1.csv')

# View the data
df.head()

,Crime ID,Month,Longitude,Latitude,Location,Crime type,Last outcome category,Context
0,70285599d05670ce933248ddc9fc8f636039e67974f77f...,2024-11,-0.114219,51.417013,On or near Covington Way,Criminal damage and arson,Investigation complete; no suspect identified,The investigation into the case of criminal da...
1,ecec6c926b9930fcdf03da4decd3bc9269153feca19713...,2024-11,-0.114219,51.417013,On or near Covington Way,Other theft,Under investigation,The investigation for this theft has been ongo...
2,e2af607ecccd83e2360e91fba01a582218fc18ed78b0e1...,2024-11,-0.110382,51.415908,On or near Norbury Hill,Vehicle crime,Investigation complete; no suspect identified,A car theft occurred on the evening of the 11t...
3,f2ce11ab13860e7b474a6cc348d8e7d788adee75ca6f46...,2024-11,-0.108812,51.420496,On or near Arnull'S Road,Violence and sexual offences,Under investigation,An incident occurred on or near Arnull's Road ...
4,c0d625a4f829e999baedec639871b55e07c0325ad91e2f...,2024-11,-0.110382,51.415908,On or near Norbury Hill,Violence and sexual offences,Under investigation,A series of reports were made citing the threa...


In [122]:
drop_cols = [
  "Reported by",
  "Falls within",
  "LSOA code",
  "LSOA name",
]

df = df.drop(columns=drop_cols, errors='ignore')
df = df[df["Context"].notna()]

print(df.shape)
df.head()

(780, 8)


,Crime ID,Month,Longitude,Latitude,Location,Crime type,Last outcome category,Context
0,70285599d05670ce933248ddc9fc8f636039e67974f77f...,2024-11,-0.114219,51.417013,On or near Covington Way,Criminal damage and arson,Investigation complete; no suspect identified,The investigation into the case of criminal da...
1,ecec6c926b9930fcdf03da4decd3bc9269153feca19713...,2024-11,-0.114219,51.417013,On or near Covington Way,Other theft,Under investigation,The investigation for this theft has been ongo...
2,e2af607ecccd83e2360e91fba01a582218fc18ed78b0e1...,2024-11,-0.110382,51.415908,On or near Norbury Hill,Vehicle crime,Investigation complete; no suspect identified,A car theft occurred on the evening of the 11t...
3,f2ce11ab13860e7b474a6cc348d8e7d788adee75ca6f46...,2024-11,-0.108812,51.420496,On or near Arnull'S Road,Violence and sexual offences,Under investigation,An incident occurred on or near Arnull's Road ...
4,c0d625a4f829e999baedec639871b55e07c0325ad91e2f...,2024-11,-0.110382,51.415908,On or near Norbury Hill,Violence and sexual offences,Under investigation,A series of reports were made citing the threa...


In [123]:
import iris
import time
import os

In [124]:
username = 'demo'
password = 'demo'
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '1972' 
namespace = 'USER'
CONNECTION_STRING = f"{hostname}:{port}/{namespace}"
print(CONNECTION_STRING)

localhost:1972/USER


In [125]:
conn = iris.connect(CONNECTION_STRING, username, password)
cursor = conn.cursor()

In [126]:
tableName = "SafeRoute.CrimeDataSample"
tableDefinition = """(CrimeID VARCHAR(100),
                      Month VARCHAR(20),
                      Longitude DOUBLE,
                      Latitude DOUBLE,
                      Location VARCHAR(255),
                      CrimeType VARCHAR(255),
                      LastOutcomeCategory VARCHAR(255),
                      Context VARCHAR(2000),
                      ContextVector VECTOR(DOUBLE, 384)) """

In [127]:
try:
    cursor.execute(f"DROP TABLE {tableName}")  
except:
    pass
cursor.execute(f"CREATE TABLE {tableName} {tableDefinition}")

0

In [128]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained sentence transformer model. This model's output vectors are of size 384
model = SentenceTransformer('all-MiniLM-L6-v2')

In [129]:
# Generate embeddings for all descriptions at once. Batch processing makes it faster
embeddings = model.encode(df['Context'].tolist(), normalize_embeddings=True)

# Add the embeddings to the DataFrame
df['Context vector'] = embeddings.tolist()

In [130]:
print(f"Shape: {df.shape}")
df.head()

Shape: (780, 9)


,Crime ID,Month,Longitude,Latitude,Location,Crime type,Last outcome category,Context,Context vector
0,70285599d05670ce933248ddc9fc8f636039e67974f77f...,2024-11,-0.114219,51.417013,On or near Covington Way,Criminal damage and arson,Investigation complete; no suspect identified,The investigation into the case of criminal da...,"[0.03810160234570503, 0.09380751848220825, -0...."
1,ecec6c926b9930fcdf03da4decd3bc9269153feca19713...,2024-11,-0.114219,51.417013,On or near Covington Way,Other theft,Under investigation,The investigation for this theft has been ongo...,"[-0.10195904225111008, 0.07268644124269485, 0...."
2,e2af607ecccd83e2360e91fba01a582218fc18ed78b0e1...,2024-11,-0.110382,51.415908,On or near Norbury Hill,Vehicle crime,Investigation complete; no suspect identified,A car theft occurred on the evening of the 11t...,"[-0.08631739765405655, 0.1327429711818695, 0.0..."
3,f2ce11ab13860e7b474a6cc348d8e7d788adee75ca6f46...,2024-11,-0.108812,51.420496,On or near Arnull'S Road,Violence and sexual offences,Under investigation,An incident occurred on or near Arnull's Road ...,"[-0.00833199918270111, 0.061875417828559875, -..."
4,c0d625a4f829e999baedec639871b55e07c0325ad91e2f...,2024-11,-0.110382,51.415908,On or near Norbury Hill,Violence and sexual offences,Under investigation,A series of reports were made citing the threa...,"[-0.060702770948410034, 0.058162324130535126, ..."


In [131]:
cursor.execute(f"DELETE FROM {tableName}")
sql = f"Insert into {tableName} (CrimeID, Month, Longitude, Latitude, Location, CrimeType, LastOutcomeCategory, Context, ContextVector) values (?,?,?,?,?,?,?,?,TO_VECTOR(?))"
start_time = time.time()

data = [(
    row['Crime ID'],
    row['Month'],
    row['Longitude'],
    row['Latitude'],
    row['Location'],
    row['Crime type'],
    row['Last outcome category'],
    row['Context'],
    str(row['Context vector'])
    )
    for index,row in df.iterrows()
]
cursor.executemany(sql, data)
end_time = time.time()
print(f"Time taken to add {len(df)} entries: {end_time-start_time} seconds")

Time taken to add 780 entries: 0.7510364055633545 seconds


In [132]:
# cursor.execute(f"""
#   ALTER TABLE {tableName}
#   DROP COLUMN Context
#   """)